# n-gram 언어모델
단어 대신 아이템 넣어서 아이템 바스켓(문장) 생성

In [1]:
#코드 참조:
#데이터 사이언스 스쿨: 확률론적 언어 모형(https://datascienceschool.net/view-notebook/a0c848e1e2d343d685e6077c35c4203b/)
from nltk import bigrams, word_tokenize
from nltk.util import ngrams
import pandas as pd
import numpy as np
from nltk import ConditionalFreqDist
import pickle
import random

In [41]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

## 아이템셋 토큰화
- window 사이즈 2인 n-gram 모형
- SS: 문장의 처음
- SE: 문장의 끝

In [55]:
with open('toy_splitted_movies.txt', 'rb') as f:
    splitted_movies = pickle.load(f)

In [56]:
splitted_movies

[['110', '457'],
 ['1', '110', '296', '457', '588'],
 ['1', '457', '588'],
 ['1', '110', '527'],
 ['50', '296'],
 ['50', '110', '457'],
 ['110', '296', '457', '588'],
 ['1', '50', '296'],
 ['50', '527', '588'],
 ['1', '50', '110', '588'],
 ['1', '50', '296', '527'],
 ['1', '50'],
 ['1', '50', '110', '527'],
 ['50', '457', '527'],
 ['296', '588'],
 ['296', '457'],
 ['110', '296'],
 ['1', '588'],
 ['50', '110', '457', '588'],
 ['1', '110'],
 ['1', '296', '527', '588'],
 ['110', '527', '588'],
 ['1', '110', '457'],
 ['1', '50', '110', '296', '527', '588'],
 ['527', '588'],
 ['1', '296', '457', '527', '588'],
 ['50', '110', '527'],
 ['1', '50', '110', '296', '457', '588'],
 ['296', '457', '527'],
 ['1', '50', '457', '527', '588'],
 ['1', '50', '527', '588'],
 ['50', '588'],
 ['1', '50', '110'],
 ['50', '457', '527', '588'],
 ['1', '296', '457', '588'],
 ['50', '110', '296', '457', '527'],
 ['110', '296', '588'],
 ['110', '296', '457', '527', '588'],
 ['110', '296', '457'],
 ['1', '296', '4

In [57]:
# 아이템 1개 이하 유저 거르기
for i in range(len(splitted_movies)):
    if len(splitted_movies[i]) < 2:
        print(i)
        print(splitted_movies[i])

In [58]:
sentences = []
for tokens in splitted_movies:
    bigram = ngrams(tokens, 2, pad_left=True, pad_right=True, left_pad_symbol="SS", right_pad_symbol="SE")
    sentences += [t for t in bigram]

In [59]:
print(sentences)

[('SS', '110'), ('110', '457'), ('457', 'SE'), ('SS', '1'), ('1', '110'), ('110', '296'), ('296', '457'), ('457', '588'), ('588', 'SE'), ('SS', '1'), ('1', '457'), ('457', '588'), ('588', 'SE'), ('SS', '1'), ('1', '110'), ('110', '527'), ('527', 'SE'), ('SS', '50'), ('50', '296'), ('296', 'SE'), ('SS', '50'), ('50', '110'), ('110', '457'), ('457', 'SE'), ('SS', '110'), ('110', '296'), ('296', '457'), ('457', '588'), ('588', 'SE'), ('SS', '1'), ('1', '50'), ('50', '296'), ('296', 'SE'), ('SS', '50'), ('50', '527'), ('527', '588'), ('588', 'SE'), ('SS', '1'), ('1', '50'), ('50', '110'), ('110', '588'), ('588', 'SE'), ('SS', '1'), ('1', '50'), ('50', '296'), ('296', '527'), ('527', 'SE'), ('SS', '1'), ('1', '50'), ('50', 'SE'), ('SS', '1'), ('1', '50'), ('50', '110'), ('110', '527'), ('527', 'SE'), ('SS', '50'), ('50', '457'), ('457', '527'), ('527', 'SE'), ('SS', '296'), ('296', '588'), ('588', 'SE'), ('SS', '296'), ('296', '457'), ('457', 'SE'), ('SS', '110'), ('110', '296'), ('296', 'S

In [60]:
movieId_to_name = pd.Series(movies.title.values, index = movies.movieId.values).to_dict()
name_to_movieId = pd.Series(movies.movieId.values, index = movies.title).to_dict()

In [61]:
cfd = ConditionalFreqDist(sentences)

In [62]:
from nltk.probability import ConditionalProbDist, MLEProbDist
cpd = ConditionalProbDist(cfd, MLEProbDist)

In [63]:
def sentence_score(s):
    p = 0.0
    for i in range(len(s) - 1):
        c = s[i]
        w = s[i + 1]
        p += np.log(cpd[c].prob(w) + np.finfo(float).eps)
    return np.exp(p)

In [64]:
def generate_sentence(seed=None, start_word="SS"):
    if seed is not None:
        import random
        random.seed(seed)
        
    c = start_word
    sentence = []
    sentence.append(c)
    
    while True:
        if c not in cpd:
            break
        w = cpd[c].generate()
        if w == "SE":
            break
        else:
            w2 = w
        sentence.append(w2)
        c = w
    
    moviename = []
    for i in sentence:
        mname = movieId_to_name[int(i)]
        moviename.append(mname)
    
    return sentence, moviename

In [65]:
# 모든 유저에 대해 csv만들기
df = pd.DataFrame(columns = ['movie_candidates'])
for i in range(len(splitted_movies)):
    sentence, movie = generate_sentence(start_word=splitted_movies[i][0])
    df.loc[i] = [sentence]

In [66]:
df.to_csv('ngram_candidates_small.csv')

In [67]:
df

,movie_candidates
0,"[110, 296, 457, 527]"
1,"[1, 296, 527, 588]"
2,"[1, 296]"
3,"[1, 50, 296, 457, 588]"
4,"[50, 296, 527, 588]"
...,...
115,"[1, 50, 457, 588]"
116,"[50, 296, 457, 527]"
117,[457]
118,"[50, 527, 588]"


## 개선해야할 점
- 조건부 확률... 순서는 상관이 없기 때문에 이 점 개선
- 출발 아이템을 frequent한 것으로 바꿔주기

In [26]:
def find_usr_with_id(mid):
    usr = []
    for i in splitted_movies:
        if mid in i:
            usr.append(i)
    
    # 해당 mid를 본 유저와 생성한 sentence를 비교
    compare = [] # 생성한 sentence와 비교해 일치율 구하기
    for u in usr:
        c = list(set(u).intersection(sentence))
        prob = (len(c) / len(sentence)) * 100
        compare.append(prob)
    
    return usr, compare

In [27]:
usr, compare = find_usr_with_id("260")

In [36]:
compare.index(max(compare))

5

In [37]:
compare

[33.33333333333333,
 66.66666666666666,
 66.66666666666666,
 66.66666666666666,
 66.66666666666666,
 100.0,
 100.0,
 100.0,
 33.33333333333333,
 66.66666666666666,
 100.0,
 66.66666666666666,
 100.0,
 100.0,
 100.0,
 33.33333333333333,
 33.33333333333333,
 66.66666666666666,
 66.66666666666666,
 33.33333333333333,
 66.66666666666666,
 66.66666666666666,
 66.66666666666666,
 66.66666666666666,
 66.66666666666666,
 33.33333333333333,
 100.0,
 66.66666666666666,
 66.66666666666666,
 66.66666666666666,
 33.33333333333333]

In [23]:
# support 실제 얼마나 나오는지
# 출발도 frequent한 아이템을 가지고 real support 구하기
# association rule로 구하는 것 보다 이게 리즈너블 한가?

In [ ]:
# 순서 정렬해서 돌리기
# association에서는 실제로 